In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  scipy.stats import zscore
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [9]:
from sklearn import linear_model

sckit-learn kütüphanesinde sabit ekleyip regresyon hesaplarken; statsmodel.api kütüphanesinde sabit eklenmemektedir.

# Soru 1:
Now, it's your turn to design your first regression model. In this module, you'll be using a version of the "House Prices" dataset from Kaggle.
* Investigate the data and do any necessary data cleaning.

In [6]:
# Öncelikle 3 farklı csv dosyasını okuyup inceleyelim
test=pd.read_csv("test.csv")
display("Test dataset:",test.head(3))

train=pd.read_csv("train.csv")
display("Train dataset:",train.head(3))

sample_submission=pd.read_csv("sample_submission.csv")
display("Sample submission dataset:",sample_submission.head(3))


# Ardndan "test" datasetiyle "sample_submission" datasetini mergeleyelim
merge_data=test.merge(sample_submission, left_on="Id", right_on="Id")
display("Test & Sample submission merged dataset:",merge_data.head(3))


# Sonra aynı kolonlara sahip olan train datasıyla sütunlar ortak olacak şekilde concat fonksiyonunu kullanalım.
all_data=pd.concat([train,merge_data])
display("All dataset:",all_data.head(3))


'Test dataset:'

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal


'Train dataset:'

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


'Sample submission dataset:'

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570


'Test & Sample submission merged dataset:'

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,169277.052498
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,187758.393989
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,183583.683570


'All dataset:'

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0


In [126]:
all_data.isna().sum().head()

Id             0
MSSubClass     0
MSZoning       4
LotFrontage    0
LotArea        0
dtype: int64

In [125]:
# Missing valuelardan kurtulalım:
all_data["LotFrontage"].fillna(all_data["LotFrontage"].mode()[0], inplace=True)
all_data["Utilities"].fillna(all_data["Utilities"].mode()[0], inplace=True)
all_data["Exterior1st"].fillna(all_data["Exterior1st"].mode()[0], inplace=True)
all_data["Exterior2nd"].fillna(all_data["Exterior2nd"].mode()[0], inplace=True)
all_data["GarageYrBlt"].fillna(all_data["GarageYrBlt"].mode()[0], inplace=True)
all_data["GarageCars"].fillna(all_data["GarageCars"].mode()[0], inplace=True)
all_data["GarageArea"].fillna(all_data["GarageArea"].mode()[0], inplace=True)
all_data["SaleType"].fillna(all_data["SaleType"].mode()[0], inplace=True)


Hedef değişkenim SalePrice olduğu için, hedef değişkenimi açıklamada kullanacağım özellikler; centalair, overallcond, utilities olabilir. Daha sonra bu özelliklerimin regresyon modelindeki katsayılarıyla saleprice ı tahmin etmeye çalışacağım.

# Soru 2:
Explore the data and find some variables that you think would be useful in the prediction of the house prices.

In [117]:
all_data.isna().sum()

Id                   0
MSSubClass           0
MSZoning             4
LotFrontage          0
LotArea              0
Street               0
Alley             2721
LotShape             0
LandContour          0
Utilities            0
LotConfig            0
LandSlope            0
Neighborhood         0
Condition1           0
Condition2           0
BldgType             0
HouseStyle           0
OverallQual          0
OverallCond          0
YearBuilt            0
YearRemodAdd         0
RoofStyle            0
RoofMatl             0
Exterior1st          0
Exterior2nd          0
MasVnrType          24
MasVnrArea          23
ExterQual            0
ExterCond            0
Foundation           0
                  ... 
KitchenQual          1
TotRmsAbvGrd         0
Functional           2
Fireplaces           0
FireplaceQu       1420
GarageType         157
GarageYrBlt          0
GarageFinish       159
GarageCars           0
GarageArea           0
GarageQual         159
GarageCond         159
PavedDrive 

In [129]:
# hedef değişkeniyle olan korelasyonu inceleyelim:
all_data.corr()["SalePrice"].reset_index()

# Korelasyon tablosundan gördüğümüz saleprice ile 1 e yakın ilişki içinde bulunan değişkenler:
# OverallQual, GrLivArea, GarageCars, GarageArea  olurdu.

,index,SalePrice
0,Id,-0.011630
1,MSSubClass,-0.087910
2,LotFrontage,0.294727
3,LotArea,0.296497
4,OverallQual,0.550911
5,OverallCond,-0.065785
6,YearBuilt,0.362066
7,YearRemodAdd,0.350032
8,MasVnrArea,0.355608
9,BsmtFinSF1,0.272202


# Soru 3:
Build your initial model using these features and estimate the parameters using OLS.

In [127]:
# Regresyon modelinde kullanacağım hedef değişkenler ve özelliklerimi içeren daha az sayıdaki bir dataframe oluşturalım:

# Hedef değişkenim: "SalePrice"
# Özelliklerim: "central_air", "OverallCond", "Utilities"

#linear regresyonu modeli için sklerarn kütüphanesindeli linearmodel modülünü yükleyelim:
from sklearn import linear_model  

In [128]:
# pd.get_dummies() ile kategorik değişkenleri sürekli değişkenlere dönüştürelim

all_data["CentralAir"].unique()

all_data["central_air"]=pd.get_dummies(all_data["CentralAir"], drop_first=True)
all_data["central_air"].reset_index().head(5)

,index,central_air
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [130]:
all_data["utilities_cont"]=pd.get_dummies(all_data["Utilities"], drop_first=True)
all_data["utilities_cont"].reset_index().head()

,index,utilities_cont
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [131]:
# Hedef değişken ve özellikleri tanımlayalım:

Y=all_data["SalePrice"]
X=all_data[["central_air", "OverallCond","utilities_cont"]]

In [132]:
# Linear regresyon modelinde değişkenleri fit edelim:

lrm=linear_model.LinearRegression()
lrm.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [133]:
# Regresyon sonucunun değişkenlerini yazdıralım:

print("Features: ", lrm.coef_)
print("Bias: ", lrm.intercept_)

Features:  [ 44195.29314249  -4579.66525526 -43541.22879799]
Bias:  164323.92718704877


In [ ]:
# Formül:

Y= 164348.66 + (44181.91)*x - (-4584.54)*z - (-43541.22)* t

# Y hedef değişkenini tahmin etmek için x,z yada t özelliklerinden biri sabit tutulup her bir özellik için Y değeri hesaplanır.
# Formülden de görebileceğimiz gibi Y hedef değişkeni ile x, pozitif korele; z ile t ise negatif ilişkilidir.

In [134]:
# regresyon sonucunu yorumlamak için statsmodels.api kütüphanesini import edelim:

import statsmodels.api as sm 

In [135]:
X= sm.add_constant(X)
results = sm.OLS(Y,X).fit() # en küçük kareler yöntemini(OLS) uygulayalım
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     41.69
Date:                Sun, 06 Sep 2020   Prob (F-statistic):           2.27e-26
Time:                        17:22:03   Log-Likelihood:                -36065.
No. Observations:                2919   AIC:                         7.214e+04
Df Residuals:                    2915   BIC:                         7.216e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.643e+05   6236.246     26.350      0.000    1.52e+05    1.77e+05
central_air      4.42e+04   4187.922     10.553      0.000     3.6e+04    5.24e+04
OverallCond    -4579.6653    941.782     -4.863      0.000   -6426.292   -2733.039
utilities_cont -4.354e+04   5.62e+04     -0.774      0.439   -1.54e+05    6.67e+04
==============================================================================
Omnibus:                     1809.228   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            31433.475
Skew:                           2.633   Prob(JB):                         0.00
Kurtosis:                      18.189   Cond. No.                         315.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""